# Core Python praktika duomenų analitikai

Šiame faile pateikiami baziniai „core Python“ įgūdžiai, kurie dažniausiai reikalingi prieš pradedant darbą su `NumPy` ir `pandas`.

Svarbiausios temos:
- Kintamieji, tipai, konvertavimas, `None`, `NaN` idėja
- Darbas su `str`, `list`, `dict`, `tuple`, `set`
- Valdymo logika: `if`, `for`, `while`
- Dažniausios „built-in“ funkcijos ir praktiniai pritaikymai analitikoje
- Rikiavimas, filtravimas, grupavimas be `pandas`
- Funkcijos: `def`, `return`, „docstring“, `*args`, `**kwargs`
- `lambda`, `map`, `filter`, `sorted(key=...)`
- Klaidos ir išimtys: `try/except`
- Gerosios praktikos ir dažnos klaidos

Pastaba: pavyzdžiai parinkti taip, kad būtų lengvai perkeliami į `NumPy` / `pandas` darbo eigą.

## 1. Minimalus „analitinis“ duomenų pavyzdys

Vietoje didelių duomenų failų naudojamas mažas sąrašas su „eilutėmis“ (įrašais). Tai panašu į `pandas` DataFrame logiką: kiekvienas įrašas turi laukus, o analizė dažnai reiškia filtravimą, agregavimą ir transformacijas.

In [1]:
import pandas as pd
import numpy as np

orders = [
    {"order_id": 1001, "customer": "Asta",   "country": "LT", "category": "Food",   "amount": 19.90, "items": 2, "returned": False},
    {"order_id": 1002, "customer": "Mantas", "country": "LV", "category": "Tech",   "amount": 249.00, "items": 1, "returned": False},
    {"order_id": 1003, "customer": "Asta",   "country": "LT", "category": "Food",   "amount": 5.50,  "items": 1, "returned": True},
    {"order_id": 1004, "customer": "Ieva",   "country": "EE", "category": "Home",   "amount": 39.99, "items": 3, "returned": False},
    {"order_id": 1005, "customer": "Tomas",  "country": "LT", "category": "Tech",   "amount": 99.00, "items": 1, "returned": False},
    {"order_id": 1006, "customer": "Ieva",   "country": "EE", "category": "Food",   "amount": 12.00, "items": 2, "returned": False},
]

len(orders), orders[0]
df = pd.DataFrame(orders)
df


,order_id,customer,country,category,amount,items,returned
0,1001,Asta,LT,Food,19.90,2,False
1,1002,Mantas,LV,Tech,249.00,1,False
2,1003,Asta,LT,Food,5.50,1,True
3,1004,Ieva,EE,Home,39.99,3,False
4,1005,Tomas,LT,Tech,99.00,1,False
5,1006,Ieva,EE,Food,12.00,2,False


## 2. Tipai, konvertavimas ir „missing values“ idėja

Duomenų analitikoje svarbu atpažinti:
- kada reikšmė yra `None` (trūksta duomenų),
- kada reikšmė yra tekstas, nors turėtų būti skaičius,
- kada atsiranda „negalimos“ reikšmės (pvz., dalyba iš nulio).

`pandas` dažnai naudoja `NaN`, tačiau „core Python“ lygmenyje dažnai pakanka tvarkyti `None` ir suvaldyti klaidas.

In [2]:
value_text = "249.00"
value_num = float(value_text)

missing = None
o = 1
type(value_text), type(value_num), missing is None, o


(str, float, True, 1)

In [3]:
# Dažna klaida: bandymas sudėti tekstą su skaičiumi
try:
    result = "10" + 5
except TypeError as e:
    result = f"Klaida: {e}"

result


'Klaida: can only concatenate str (not "int") to str'

Geroji praktika:
- konvertavimą daryti aiškiai: `int(...)`, `float(...)`, `str(...)`;
- naudoti `try/except`, kai duomenys gali būti netvarkingi.

In [4]:
def safe_float(x):
    """Grąžina float arba None, jei konvertuoti nepavyksta."""
    try:
        return float(x)
    except (TypeError, ValueError):
        return None

safe_float("12.5"), safe_float("abc"), safe_float(None), safe_float(12)


(12.5, None, None, 12.0)

## 3. Dažniausios „built-in“ funkcijos analitikoje

Ypač dažnai naudojamos:
- `len`, `sum`, `min`, `max`
- `sorted`, `round`
- `enumerate`, `zip`
- `any`, `all`
- `set` (unikalioms reikšmėms)

Svarbu suprasti, kad daugumą veiksmų galima atlikti „core Python“ lygmenyje, o vėliau tas pats principas perkeliama į `pandas`.

In [5]:
len(orders), orders[0]


(6,
 {'order_id': 1001,
  'customer': 'Asta',
  'country': 'LT',
  'category': 'Food',
  'amount': 19.9,
  'items': 2,
  'returned': False})

In [6]:
amounts = [o["amount"] for o in orders]
total_revenue = sum(amounts)
min_amount = min(amounts)
max_amount = max(amounts)

total_revenue, min_amount, max_amount, len(amounts)


(425.39, 5.5, 249.0, 6)

In [7]:
# Unikalios kategorijos ir šalys
categories = sorted([o["category"] for o in orders])
countries = sorted({o["country"] for o in orders})

categories, countries


(['Food', 'Food', 'Food', 'Home', 'Tech', 'Tech'], ['EE', 'LT', 'LV'])

## 4. Darbas su `str`: valymas ir standartizavimas

Duomenyse tekstai dažnai turi:
- tarpus pradžioje / pabaigoje,
- skirtingą raidžių dydį,
- „netyčinius“ simbolius.

Naudingi metodai:
- `.strip()`, `.lower()`, `.upper()`, `.replace()`, `.split()`
- `f-string` formatavimas: `f"...{kintamasis}..."`

In [40]:
raw_city = "  Vilnius  "
clean_city = raw_city.strip().upper()

raw_city, clean_city


('  Vilnius  ', 'VILNIUS')

In [41]:
customer = orders[0]["customer"]
amount = orders[0]["amount"]

msg = f"Klientas: {customer}, suma: {amount:.2f} EUR"
msg, print(msg)


Klientas: Asta, suma: 19.90 EUR


('Klientas: Asta, suma: 19.90 EUR', None)

Dažna klaida:
- palikti tekstus „kaip yra“ ir vėliau gauti dublikatus (pvz., `Vilnius`, `vilnius`, ` VILNIUS `).

## 5. `list` ir `dict`: bazinės operacijos analitikoje

Analitinis mąstymas dažnai remiasi:
- filtravimu (atrinkti įrašus pagal sąlygą),
- transformacijomis (sukurti naują lauką),
- agregavimu (sumos, vidurkiai, skaičiai).

Žemiau pateikiami pavyzdžiai be `pandas`, kad būtų aiški logika.

In [43]:
# Filtravimas: tik negrąžinti užsakymai
not_returned = [o for o in orders if not o["returned"]]

len(not_returned), [o["order_id"] for o in not_returned]


(5, [1001, 1002, 1004, 1005, 1006])

In [48]:
# Transformacija: pridedamas laukas 'amount_per_item'
orders_enriched = []
for o in orders:
    amount_per_item = o["amount"] / o["items"]  # prielaida: items > 0
    new_o = {**o, "amount_per_item": round(amount_per_item, 2)}
    orders_enriched.append(new_o)

orders_enriched[0]


{'order_id': 1001,
 'customer': 'Asta',
 'country': 'LT',
 'category': 'Food',
 'amount': 19.9,
 'items': 2,
 'returned': False,
 'amount_per_item': 9.95}

In [49]:
print(orders_enriched)

[{'order_id': 1001, 'customer': 'Asta', 'country': 'LT', 'category': 'Food', 'amount': 19.9, 'items': 2, 'returned': False, 'amount_per_item': 9.95}, {'order_id': 1002, 'customer': 'Mantas', 'country': 'LV', 'category': 'Tech', 'amount': 249.0, 'items': 1, 'returned': False, 'amount_per_item': 249.0}, {'order_id': 1003, 'customer': 'Asta', 'country': 'LT', 'category': 'Food', 'amount': 5.5, 'items': 1, 'returned': True, 'amount_per_item': 5.5}, {'order_id': 1004, 'customer': 'Ieva', 'country': 'EE', 'category': 'Home', 'amount': 39.99, 'items': 3, 'returned': False, 'amount_per_item': 13.33}, {'order_id': 1005, 'customer': 'Tomas', 'country': 'LT', 'category': 'Tech', 'amount': 99.0, 'items': 1, 'returned': False, 'amount_per_item': 99.0}, {'order_id': 1006, 'customer': 'Ieva', 'country': 'EE', 'category': 'Food', 'amount': 12.0, 'items': 2, 'returned': False, 'amount_per_item': 6.0}]


Dažna klaida:
- dalyba iš nulio (`items = 0`) arba trūkstamos reikšmės (`None`).
Tokiais atvejais reikalinga papildoma apsauga.

In [50]:
def safe_divide(a, b):
    """Dalyba su apsauga: grąžina None, jei dalyba negalima."""
    try:
        if b in (0, None):
            return None
        if a is None:
            return None
        return a / b
    except TypeError:
        return None

safe_divide(10, 2), safe_divide(10, 0), safe_divide(None, 2), safe_divide(10, "x")


(5.0, None, None, None)

## 6. Ciklai: `for`, `while`, `break`, `continue`

Duomenų analitikoje `for` ciklas dažniausiai naudojamas:
- pereiti per įrašus,
- skaičiuoti agregatus,
- kurti naujas struktūras.

`while` naudojamas rečiau, bet jis naudingas, kai kartojama iki sąlygos įvykdymo (pvz., „kol randama klaida“).

In [52]:
# Agregavimas: pajamos pagal kategoriją (be pandas)
revenue_by_category = {}

for o in orders:
    cat = o["category"]
    print(cat)
    revenue_by_category[cat] = revenue_by_category.get(cat, 0) + o["amount"]

revenue_by_category


Food
Tech
Food
Home
Tech
Food


{'Food': 37.4, 'Tech': 348.0, 'Home': 39.99}

In [15]:
# 'continue' pavyzdys: praleisti grąžintus užsakymus
revenue_without_returns = 0.0

for o in orders:
    if o["returned"]:
        continue
    revenue_without_returns += o["amount"]

revenue_without_returns


419.89

In [16]:
# 'while' pavyzdys: rasti pirmą užsakymą, kurio suma viršija slenkstį
threshold = 100
i = 0
first_big = None

while i < len(orders):
    if orders[i]["amount"] > threshold:
        first_big = orders[i]
        break
    i += 1

first_big


{'order_id': 1002,
 'customer': 'Mantas',
 'country': 'LV',
 'category': 'Tech',
 'amount': 249.0,
 'items': 1,
 'returned': False}

Geroji praktika:
- jei užtenka `for`, dažniausiai nereikia `while`;
- ciklų viduje vengti sudėtingos logikos, geriau išskirti į funkcijas.

## 7. „Comprehensions“: trumpesnė ir aiški logika

`list` / `dict` / `set` comprehensions leidžia užrašyti tą patį glaustai.
Svarbu nepersistengti: jei sakinys tampa neįskaitomas, geriau grįžti prie įprasto `for`.

In [53]:
# Sąrašas: sumos tik iš LT
lt_amounts = [o["amount"] for o in orders if o["country"] == "LT"]
lt_amounts, sum(lt_amounts)


([19.9, 5.5, 99.0], 124.4)

In [18]:
# Žodynas: užsakymų skaičius pagal šalį
count_by_country = {c: 0 for c in {o["country"] for o in orders}}
for o in orders:
    count_by_country[o["country"]] += 1

count_by_country


{'LV': 1, 'EE': 2, 'LT': 3}

## 8. Rikiavimas ir `key=`

Duomenų analitikoje labai dažnas veiksmas:
- išrikiuoti pagal sumą,
- rasti TOP N,
- išrikiuoti pagal kelis laukus.

Svarbu mokėti naudoti:
- `sorted(iterable, key=..., reverse=...)`
- `.sort(...)` (keičia sąrašą vietoje)

In [19]:
# TOP 3 užsakymai pagal sumą
top3 = sorted(orders, key=lambda o: o["amount"], reverse=True)[:3]
[(o["order_id"], o["amount"]) for o in top3]


[(1002, 249.0), (1005, 99.0), (1004, 39.99)]

In [20]:
# Rikiavimas pagal kelis laukus: šalis, tada suma mažėjančiai
sorted_multi = sorted(orders, key=lambda o: (o["country"], -o["amount"]))
[(o["country"], o["order_id"], o["amount"]) for o in sorted_multi]


[('EE', 1004, 39.99),
 ('EE', 1006, 12.0),
 ('LT', 1005, 99.0),
 ('LT', 1001, 19.9),
 ('LT', 1003, 5.5),
 ('LV', 1002, 249.0)]

Dažna klaida:
- naudoti `sorted(...)` ir tikėtis, kad originalus sąrašas pasikeis.
`sorted(...)` grąžina naują sąrašą, o `.sort(...)` keičia esamą.

In [21]:
nums = [3, 1, 2]
sorted_nums = sorted(nums)

nums, sorted_nums


([3, 1, 2], [1, 2, 3])

## 9. Funkcijos su `def`: aiškumas, pernaudojimas, testavimas

Analitiniuose scenarijuose funkcijos dažnai naudojamos:
- duomenų valymui,
- metrikų skaičiavimui,
- pasikartojančios logikos iškėlimui iš ciklų.

Geroji praktika:
- aiškūs pavadinimai,
- `return` visada grąžina vieną aiškią reikšmę,
- trumpas aprašas („docstring“),
- nedaryti „slaptų“ globalių priklausomybių.

In [22]:
def total_amount(data, include_returns=False):
    """Apskaičiuoja bendrą sumą. Pagal nutylėjimą grąžinti užsakymai neįtraukiami."""
    total = 0.0
    for o in data:
        if (not include_returns) and o["returned"]:
            continue
        total += o["amount"]
    return total

total_amount(orders), total_amount(orders, include_returns=True)


(419.89, 425.39)

In [23]:
def revenue_by_key(data, key_name):
    """Sugrupuoja pajamas pagal pasirinktą lauką (pvz., 'country' arba 'category')."""
    out = {}
    for o in data:
        k = o[key_name]
        out[k] = out.get(k, 0) + o["amount"]
    return out

revenue_by_key(orders, "country"), revenue_by_key(orders, "category")


({'LT': 124.4, 'LV': 249.0, 'EE': 51.99},
 {'Food': 37.4, 'Tech': 348.0, 'Home': 39.99})

## 10. `*args` ir `**kwargs`: kada praverčia

Tai naudinga, kai:
- funkcija priima kintamą skaičių argumentų,
- norima perduoti „parametrų rinkinį“ į kitą funkciją.

Analitikoje dažnai pasitaiko kuriant „wrapper“ funkcijas (pvz., duomenų validacijai ar formatavimui).

In [24]:
def mean_of(*values):
    """Grąžina aritmetinį vidurkį iš pateiktų reikšmių."""
    if not values:
        return None
    return sum(values) / len(values)

mean_of(1, 2, 3), mean_of()


(2.0, None)

In [25]:
def format_currency(amount, **kwargs):
    """Formatuoja sumą į tekstą. Leidžia valdyti valiutos kodą ir skaičių po kablelio."""
    currency = kwargs.get("currency", "EUR")
    decimals = kwargs.get("decimals", 2)
    if amount is None:
        return None
    return f"{amount:.{decimals}f} {currency}"

format_currency(19.9), format_currency(19.9, currency="USD", decimals=0)


('19.90 EUR', '20 USD')

## 11. `lambda`, `map`, `filter`: kada naudoti

Svarbu suprasti:
- `lambda` yra „trumpa funkcija“, dažnai naudojama `key=` argumente (rikiavimui) arba paprastoms transformacijoms.
- `map` ir `filter` yra alternatyva comprehension'ams, bet dažnai comprehension'ai yra skaitomesni.

Geroji praktika:
- jei logika sudėtingesnė nei viena eilutė, naudoti `def`.

In [26]:
# map: ištraukti sumas
amounts_map = list(map(lambda o: o["amount"], orders))
amounts_map


[19.9, 249.0, 5.5, 39.99, 99.0, 12.0]

In [27]:
# filter: atrinkti Tech kategoriją
tech_orders = list(filter(lambda o: o["category"] == "Tech", orders))
[(o["order_id"], o["amount"]) for o in tech_orders]


[(1002, 249.0), (1005, 99.0)]

## 12. `enumerate` ir `zip`: tvarkingas iteravimas

- `enumerate` praverčia, kai reikalingas indeksas.
- `zip` praverčia, kai reikia eiti per kelis sąrašus vienu metu (pvz., datas ir reikšmes).

In [28]:
for idx, o in enumerate(orders[:3], start=1):
    print(idx, o["order_id"], o["amount"])


1 1001 19.9
2 1002 249.0
3 1003 5.5


In [29]:
dates = ["2026-01-01", "2026-01-02", "2026-01-03"]
values = [10, 12, 9]

pairs = list(zip(dates, values))
pairs


[('2026-01-01', 10), ('2026-01-02', 12), ('2026-01-03', 9)]

## 13. Paprastos metrikos be `pandas`: vidurkis, mediana, dispersija

Prieš `pandas` patogu mokėti minimaliai paskaičiuoti:
- vidurkį, medianą,
- standartinį nuokrypį,
- procentilius (supaprastintai).

Tam galima naudoti `statistics` modulį.

In [30]:
import statistics as stats

amounts = [o["amount"] for o in orders if not o["returned"]]

mean_ = stats.mean(amounts)
median_ = stats.median(amounts)
stdev_ = stats.pstdev(amounts)  # populiacijos standartinis nuokrypis

mean_, median_, stdev_


(83.978, 39.99, 87.9541786159134)

In [31]:
import math

# Paprastas procentilio įvertinimas (be numpy):
def percentile(values, p):
    """Grąžina p procentilį (0-100), naudodama paprastą 'nearest rank' metodą."""
    if not values:
        return None
    if not (0 <= p <= 100):
        raise ValueError("p turi būti intervale 0..100")
    vals = sorted(values)
    k = math.ceil((p / 100) * len(vals))
    k = max(1, k)
    return vals[k - 1]

percentile(amounts, 50), percentile(amounts, 90)


(39.99, 249.0)

dazniausia klaida - nesuimportuoti moduliu :D import math

Dažna klaida:
- maišyti populiacijos ir imties standartinį nuokrypį.
`statistics.pstdev` (population) ir `statistics.stdev` (sample) duoda skirtingus rezultatus.

In [32]:
stats.pstdev(amounts), stats.stdev(amounts)


(87.9541786159134, 98.33576114517038)

## 14. „Group by“ logika be `pandas`: skaičius ir vidurkis pagal grupę

Tai tiesioginė įžanga į `pandas.groupby(...)` mąstymą.

In [33]:
# Skaičius ir pajamos pagal klientą
agg_by_customer = {}  # customer -> {"count": ..., "revenue": ...}

for o in orders:
    cust = o["customer"]
    if cust not in agg_by_customer:
        agg_by_customer[cust] = {"count": 0, "revenue": 0.0}
    agg_by_customer[cust]["count"] += 1
    agg_by_customer[cust]["revenue"] += o["amount"]

agg_by_customer


{'Asta': {'count': 2, 'revenue': 25.4},
 'Mantas': {'count': 1, 'revenue': 249.0},
 'Ieva': {'count': 2, 'revenue': 51.99},
 'Tomas': {'count': 1, 'revenue': 99.0}}

In [34]:
# Vidutinė užsakymo suma pagal klientą
avg_by_customer = {k: v["revenue"] / v["count"] for k, v in agg_by_customer.items()}
avg_by_customer


{'Asta': 12.7, 'Mantas': 249.0, 'Ieva': 25.995, 'Tomas': 99.0}

## 15. Išimtys ir klaidų diagnostika

Duomenų failuose dažnai pasitaiko:
- neteisingi tipai,
- trūkstamos reikšmės,
- neteisingi formatavimai.

`try/except` leidžia suvaldyti situaciją ir tęsti analizę, bet svarbu nepiktnaudžiauti „tyliu“ klaidų slėpimu.

In [35]:
dirty_amounts = ["19.90", "12", None, "abc", "99.00"]

clean = []
errors = 0

for x in dirty_amounts:
    fx = safe_float(x)
    if fx is None:
        errors += 1
        continue
    clean.append(fx)

clean, errors


([19.9, 12.0, 99.0], 2)

Geroji praktika:
- kaupti klaidų skaitiklį arba logą, kad būtų aišku, kiek įrašų „prarasta“;
- neslėpti klaidų visiškai, ypač jei tai svarbu verslo metrikoms.

## 16. Mini užduotys 

Užduotys parinktos taip, kad tiesiogiai atkartotų tipines `pandas` operacijas:
- filtruoti įrašus,
- sukurti naują lauką,
- paskaičiuoti TOP N,
- sugrupuoti ir paskaičiuoti metrikas.

In [36]:
# 1) Atrinkti LT užsakymus ir apskaičiuoti jų bendrą sumą (be grąžintų).
lt_orders = [o for o in orders if o["country"] == "LT" and not o["returned"]]
lt_total = sum(o["amount"] for o in lt_orders)

lt_orders, lt_total


([{'order_id': 1001,
   'customer': 'Asta',
   'country': 'LT',
   'category': 'Food',
   'amount': 19.9,
   'items': 2,
   'returned': False},
  {'order_id': 1005,
   'customer': 'Tomas',
   'country': 'LT',
   'category': 'Tech',
   'amount': 99.0,
   'items': 1,
   'returned': False}],
 118.9)

In [37]:
# 2) Sukurti sąrašą su (order_id, amount_per_item) tik tiems, kurių items >= 2.
pairs = []
for o in orders:
    if o["items"] >= 2:
        api = safe_divide(o["amount"], o["items"])
        pairs.append((o["order_id"], None if api is None else round(api, 2)))

pairs


[(1001, 9.95), (1004, 13.33), (1006, 6.0)]

In [38]:
# 3) Rasti TOP 2 klientus pagal pajamas (įtraukiant ir grąžintus, kad būtų paprasta logika).
rev_by_customer = revenue_by_key(orders, "customer")
top2_customers = sorted(rev_by_customer.items(), key=lambda kv: kv[1], reverse=True)[:2]

top2_customers


[('Mantas', 249.0), ('Tomas', 99.0)]

In [39]:
# 4) Paskaičiuoti grąžinimų dalį (return rate) pagal užsakymų skaičių.
total_orders = len(orders)
returned_orders = sum(1 for o in orders if o["returned"])
return_rate = returned_orders / total_orders

total_orders, returned_orders, round(return_rate, 3)


(6, 1, 0.167)

## 17. Santrauka: ką svarbu mokėti prieš `NumPy` ir `pandas`

Prieš pereinant prie `NumPy` ir `pandas`, praktikoje labiausiai atsiperka gebėjimas:
- tvarkingai dirbti su tipais ir konvertavimu (`int`, `float`, `str`, `None`),
- filtruoti ir transformuoti kolekcijas (`list`, `dict`) naudojant ciklus ir comprehension'us,
- rikiuoti duomenis su `sorted(key=...)` ir suprasti `lambda` paskirtį,
- rašyti aiškias funkcijas su `def`, naudoti `return`, turėti „docstring“,
- suvaldyti netvarkingus duomenis su `try/except` ir paprastais „safe_*“ pagalbininkais,
- suprasti „group by“ logiką (agregavimas pagal raktą), nes tai tiesiogiai pereina į `pandas.groupby`.

Dažniausios klaidos:
- neteisingi tipai (tekstas vietoje skaičiaus),
- `None` / trūkstamų reikšmių ignoravimas,
- dalyba iš nulio,
- per sudėtingos vienos eilutės `lambda` išraiškos,
- kintamųjų perrašymas ir neaiškūs pavadinimai,
- „tylus“ klaidų slėpimas be diagnostikos.